# Data formatting and Preprocessing 

## Methadology: 

1. Gather data science/data related jobs
    * General Business/Actuarial analyst positions grabbed that are not of interest
2.  Split descriptions at \n tags to gather sentences used for classification converts string gathered to list
3. Cleanse list by removing empty indexes 
4. Convert to  np array and lemmatize data 

In [97]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import nltk
from nltk import WordNetLemmatizer 
import numpy as np
import time
import joblib
import pickle
import re

In [86]:
# load saved data 
def load():
    with open('../data/job_dict.p', 'rb') as fp:
        job_dict = pickle.load(fp)
    return job_dict

def save(final_file):
    with open(f'../data/job_dict.p', 'wb') as fp:
        pickle.dump(final_file,file=fp,protocol=pickle.HIGHEST_PROTOCOL)
job_dict = load()

# 1

In [87]:
data_science = []
machine_learning = []
analyst = []
art_int = []
left_over= []
statistics= []
for listing,key in zip(job_dict,list(job_dict.keys())):
    inf = job_dict.get(key)
    role = inf.get('role')
    ds = bool(re.search(".*[Dd]ata\s.[Sc]ien.*", role))
    ml = bool(re.search(".*[Mm]achine\s[Ll]earn.*",role))
    anal = bool(re.search(".*[Dd]ata\s[Aa]nal.*", role))
    ai_long = bool(re.search(".*\s[Aa]rtificial\s[Ii]ntelligence\s.*", role))
    ai = bool(re.search(".*\s[Aa][Ii]\s.*", role))
    stat = bool(re.search(".*[St]at.*\s[Pp]rogram.*", role))
    if(ds):
        data_science.append(listing)
    elif(ml):
        machine_learning.append(listing)
    elif(anal):
        analyst.append(listing)
    elif((ai or ai_long) and listing not in data_science and listing not in analyst and listing not in machine_learning):
        art_int.append(listing)
    elif(stat and listing not in data_science and listing not in analyst and listing not in machine_learning):
        statistics.append(listing)
    else:
        left_over.append(listing)


In [88]:
print(len(analyst),len(machine_learning),len(data_science),len(statistics),len(art_int))
data = (analyst)+(machine_learning)+(data_science)+(statistics)+(art_int)


84 83 201 30 6


## 2

In [89]:
def role_get(data):
    role = []
    for i in data:
        listing= job_dict.get(i)
        role.append(listing.get('role'))
    return role
roles = role_get(data) 

## 3

In [91]:
#gets description of data 
def description_getter(data):
    descriptions=[]
    for data_point in data:
        try:
            desc = job_dict.get(data_point).get('description').split("\n")
            descriptions.append(desc)
        except AttributeError:
        #remove depreciated urls from dataset
            print(f'broken urls {data_point}')
            job_dict.pop(data_point)
            save(job_dict)
    return descriptions
segregated_data = description_getter(data)


## 4

In [92]:
#Extracts empty string list objects from data
def cleanse_empty_data(data):
   prod_data = []
   for k in data:
      [prod_data.append(x) for x in k if x !=""]
   return prod_data

prod_data = cleanse_empty_data(segregated_data)
arr_prod_data = np.array(prod_data)
lem = WordNetLemmatizer()
Lemmatizer = np.vectorize(lambda x: ' '.join([lem.lemmatize(i) for i in x.split(" ")]))
lem_data = Lemmatizer(prod_data)


## 5

In [98]:
pd.Series(lem_data).to_csv("../data/processed_data.csv")